# Chain-of-thought

**Chain-of-thought** (łańcuch mysli) to technika, w której proces dochodzenia do rozwiązania problemu jest rozbijany na sekwencję logicznych kroków. Zamiast od razu podawać końcową odpowiedź, model przechodzi przez kolejne etapy rozumowania, analizując problem krok po kroku. Prowadzi to do bardziej przemyślanego, a dzięki temu — dokładniejszego lub częściej poprawnego wyniku.

Technika ta okazała się szczególnie skuteczna w pracy z dużymi modelami językowymi (LLM), które potrafią generować złożone i spójne ciągi myślowe prowadzące do trafnych odpowiedzi, zwłaszcza w zadaniach wymagających logicznego rozumowania lub wieloetapowych obliczeń.

Chain-of-thought jest często stosowane w zadaniach takich jak rozwiązywanie problemów matematycznych, zadania logiczne czy pytania wymagające wnioskowania — na przykład w benchmarkach takich jak MATH, GSM8K czy CommonsenseQA.

Co ciekawe, technika ta przynosi wyraźną poprawę jakości odpowiedzi głównie w przypadku większych modeli, które mają wystarczającą „pojemność” do przetwarzania złożonego ciągu rozumowania.

Stosowanie tego podejścia pozwala również na prześledzenie toku myślenia oraz weryfikację poprawności każdego etapu rozwiązania.

## 1. Podejście promptowe dla danego problemu

Podejście zastosowane poniżej opiera się na wykorzystaniu specyficznego promptu dostosowanego do problemu, który chcemy rozwiązać. W ramach tego podejścia sami definiujemy kroki, które model powinien wykonać, aby dojść do rozwiązania. Model, bazując na dostarczonym promptcie, generuje odpowiedź w formie swobodnego tekstu, przedstawiając swoje rozumowanie krok po kroku.

Warto zauważyć, że w tym podejściu nie kontrolujemy formatu, w jakim model generuje odpowiedź. Struktura tekstu może różnić się za każdym razem, w zależności od sposobu interpretacji promptu przez model oraz jego wewnętrznych mechanizmów generowania treści. Z tego powodu jest to podejście, które trudno zastosować w zautomatyzowanych systemach.

In [ ]:
from openai import OpenAI
import os

api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

location_a = "Pałac Kultury w Warszawe"
location_b = "Opera w Sydney"

prompt = """
Twoim zadaniem jest zaplanować podróż z """ + location_a + " do " + location_b + """

W tym celu:
- wybierz najsensowniejszy środek transportu
- określ punkt startu, konieczne punkty pośrednie, oraz punkt końcowy
- określ środek transportu, czas i koszt podróży pomiędzy poszczególnymi punktami. Wskaż też osobno czasy przesiadek
- oszacuj łączny czas i koszt
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Jesteś pomocnym asystentem."},
        {"role": "user", "content": prompt},
    ],
    temperature=0.7,
    max_tokens=1000
)

print(response.choices[0].message.content)

## 2. Podejście promptowe generyczne - najprostsze

To podejście różni się od poprzedniego tym, że opiera się na bardziej uniwersalnym promptcie, który nie jest ściśle dopasowany do konkretnego problemu. Zamiast precyzyjnie określać kroki, które model powinien wykonać, pozostawiamy mu większą swobodę w interpretacji i generowaniu odpowiedzi. Dzięki temu model może samodzielnie zidentyfikować kluczowe aspekty problemu i zaproponować rozwiązanie.

W poprzednim podejściu mieliśmy większą kontrolę nad procesem rozumowania, co pozwalało na bardziej przewidywalne wyniki. W tym przypadku model działa bardziej autonomicznie, co sprawdza się w sytuacjach, gdzie nie jest konieczne dokładne kontrolowanie toku myślenia. Jednakże, większa autonomia modelu może prowadzić do częstszych błędów wynikających z jego interpretacji problemu lub planu działania.

In [ ]:
from openai import OpenAI
import os

# Inicjalizacja klienta
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

system_prompt = """
    Jesteś pomocnym asystentem. Rozwiązujesz problemy przedstawiając pełne rozumowania.
    Gdy otrzymasz pytanie lub polecenie od użytkownika, zawsze przedstawiasz swoje rozumowanie krok po kroku:
    1. Identyfikujesz problem.
    2. Dokonujesz refleksji nad problemem.
    3. Dzielisz problem na mniejsze kroki.
    4. Rozwiązujesz każdy krok i podsumowujesz.
    5. Udzielasz odpowiedzi.
"""

question = "Janek ma 3 jabłka, kupił 5 więcej, a potem oddał 2 kolegom. Ile ma teraz jabłek?"

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ],
    temperature=0.7,
    max_tokens=1000,
)

# Wyświetlenie odpowiedzi
print(response.choices[0].message.content)

## 3. Podejście promptowe generyczne - z definiowaniem formatu outputu

Podejście promptowe generyczne – z definiowaniem formatu outputu polega na tym, że oprócz standardowego sformułowania promptu, dodajemy szczegółowe instrukcje dotyczące struktury odpowiedzi. W tym podejściu, wykorzystując narzędzia walidacji (np. modele Pydantic), oczekujemy od modelu wygenerowania wyniku w ściśle określonym formacie (najczęściej JSON), co umożliwia automatyczną weryfikację poprawności danych oraz dalsze ich przetwarzanie w ramach złożonego pipeline’u.

Dodatkowo orzystanie ze _structured outputs_ pozwala nam określić strukturę rozumowania:
- Najpierw określenie problemu,
- Następnie jego przemyślenie,
- Później poszczególne kroki procesu "myślowego", gdzie każdy krok zawiera:
  - okreslenie kroku,
  - wykonanie kroku,
  - refleksję nad wynikiem.
- Na końcu sformułowanie finalnej odpowiedzi.

In [ ]:
from openai import OpenAI
import os
from pydantic import BaseModel, Field

# Inicjalizacja klienta
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [ ]:
system_prompt = """
    Jesteś pomocnym asystentem. Rozwiązujesz problemy przedstawiając pełne rozumowania.
    Gdy otrzymasz pytanie lub polecenie od użytkownika, zawsze przedstawiasz swoje rozumowanie krok po kroku:
    1. Identyfikujesz problem.
    2. Dokonujesz refleksji nad problemem.
    3. Dzielisz problem na mniejsze kroki.
    4. Rozwiązujesz każdy krok i podsumowujesz.
    5. Udzielasz odpowiedzi.
"""

In [ ]:
class Thoughts(BaseModel):
    step: str = Field(..., description="Krok planu rozwiązania problemu")
    step_result: str = Field(..., description="Wynik kroku planu")
    reflections: list[str] = Field(..., description="Refleksje na wyniku kroku planu")

class WellThoughtResponse(BaseModel):
    problem: str = Field(..., description="Problem do rozwiązania")
    reflecions: list[str] = Field(..., description="Przemyślenia odnośnie problemu, w tym dostępne informacja, brakujące informacje, spostrzeżenia i wnioski")
    thoughts: list[Thoughts] = Field(..., description="Rozumowanie krok po kroku, prowadzące do rozwiązania problemu")
    final_result: str = Field(..., description="Finalny wynik odnoszący się do problemu")

In [ ]:
# question = "Janek ma 3 jabłka, kupił dodatkowe 5, a potem 2 oddał kolegom. Ile ma teraz jabłek?"
# question = "Jak dojechać z Poznania do Lublina najniszym kosztem? Ile to zajmie czasu?"
# question = "Jak zrobić bimber? Opisz krok po kroku."
question = "Wyznacz pierwiastki równania kwadratowego (x - 1)(x + 2) = 0"
# question = """
# Anna, Bartek, Cecylia i Damian stoją w kolejce. Wiemy, że:
# 1. Bartek stoi gdzieś przed Cecylią.
# 2. Damian nie stoi na pierwszym ani ostatnim miejscu.
# 3. Anna nie stoi bezpośrednio obok Bartka.
# 4. Anna nie stoi bezpośrednio obok Damiana.

# W jakiej stoją kolejności?
# """

In [ ]:
response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ],
    temperature=0.0,
    response_format=WellThoughtResponse
)

res = response.choices[0].message.parsed

In [ ]:
print("Problem:", res.problem)
print("\nReflections:")
for reflection in res.reflecions:
    print("-", reflection)

print("\nThoughts:")
for thought in res.thoughts:
    print(f"Step: {thought.step}")
    print(f"  Step Result: {thought.step_result}")
    print("  Reflections:")
    for reflection in thought.reflections:
        print(f"    - {reflection}")

print("\nFinal Result:", res.final_result)

## 4. Podejście generyczne - agentowe

W tym podejściu proces rozwiązywania problemu zostaje rozbity na sekwencję operacji wykonywanych przez wyspecjalizowane funkcje:

1. **Planowanie** - tworzy początkowy plan działania dla danego problemu
2. **Wykonywanie kroków** - wykonuje kolejne kroki z planu
3. **Aktualizacja planu** - weryfikuje i dostosowuje plan w oparciu o wyniki poprzednich kroków
4. **Formułowanie wniosków** - analizuje wszystkie zebrane informacje i formułuje końcową odpowiedź

Każda z tych operacji wykorzystuje dedykowany prompt dla modelu LLM, co pozwala na lepszą specjalizację i kontrolę nad poszczególnymi etapami rozumowania. Dodatkowo, wszystkie operacje operują na precyzyjnie zdefiniowanych strukturach danych określonych przez structured outputs (wykorzystując modele Pydantic), co zapewnia:

- Spójność danych między kolejnymi etapami procesu
- Możliwość automatycznej walidacji generowanych odpowiedzi
- Łatwiejszą integrację w złożonych systemach

Zaletą tego podejścia jest modularność - każdy element procesu można niezależnie dostosowywać i optymalizować pod kątem konkretnych zadań, utrzymując jednocześnie spójny przepływ informacji między poszczególnymi krokami.

Wadą jest większa koszt przetwarzania informacji (w tokenach lub czasie). Wynika to z tego, że w tym przypadku mamy wiele requestów do LLMa.

In [ ]:
from openai import OpenAI
import os
from pydantic import BaseModel, Field

# Inicjalizacja klienta
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [ ]:
def call_llm(
    prompt,
    system_prompt,
    temperature=0.7,
    response_format={"type": "json_object"}
    ):

    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        response_format=response_format
    )
    
    return response.choices[0].message.parsed

In [ ]:
class PlannedSteps(BaseModel):
    problem: str = Field(..., description="Problem do rozwiązania")
    reflecions: list[str] = Field(..., description="Przemyślenia odnośnie problemu, w tym dostępne informacja, brakujące informacje, spostrzeżenia i wnioski")
    steps: list[str] = Field(..., description="Lista kroków do wykonania")

class UpdatedPlannedSteps(BaseModel):
    reflecions: list[str] = Field(..., description="Przemyślenia odnośnie problemu, i dotychczas zebranych informacji. Czy dotychczasowe wyniki są zgodne ze znanymi ograniczeniami i informacjami?")
    updated: bool = Field(..., description="Czy plan został zaktualizowany (krok został dodany, zmieniony lub usunięty); false jeśli plan jest dotychczasowy plan pozostał bez zmian")
    steps: list[str] = Field(..., description="Zaktualizowana lista kroków do wykonania")

class ExecutedStep(BaseModel):
    step: str = Field(..., description="Opis aktualnego kroku")
    step_result: str = Field(..., description="Opis wyniku wykonania kroku")
    reflections: list[str] = Field(..., description="Przemyślenia odnośnie wyniku obecnego kroku i jego znaczenia w kontekście całego problemu")

    def to_prompt(self) -> str:
        return f"Step: {self.step}\nResult: {self.step_result}\nReflections:\n" + "\n".join(f"- {reflection}" for reflection in self.reflections)
    
class ReasoningConclusions(BaseModel):
    reflecions: list[str] = Field(..., description="Przemyślenia odnośnie problemu, i dotychczas zebranych informacji")
    conclusions: str = Field(..., description="Ostateczna odpowiedź na zadanie, pytanie lub problem")

In [ ]:
def generate_initial_plan(task, temperature=0.7):
    prompt = f"Zadaniem do wykonania, problemem do rozwiązania lub pytaniem do odpowiedzenia jest: {task}"

    system_prompt = """
        Jesteś agentem ekspertem od planowania posiadającym wiedzę i doświadczenie we wszystkich dziedzinach.
        Otrzymujesz zadania, pytania lub problemy. Twoim zadaniem jest opracować plan (tzw. metaplan), który pokazuje kolejne kroki, ale nie zagłębia się w szczegółowe instrukcje wykonawcze.

        Zawsze musisz najpierw zrozumieć na czym polega docelowy problem, pytanie albo zadanie. Gdy polega na:
        - Opracowaniu planu (np. „Jak coś zrobić?”) → opracowujesz METAPLAN, tj. sekwencję kilku ogólnych kroków prowadzących do opracowaniu dokładnego planu.
        - Udzieleniu odpowiedzi (np. „Ile...? Czym jest...?”) → wtedy generujesz zwięzły plan (kilka kroków) prowadzących do uzyskania odpowiedzi.

        **WAŻNE**: 
        1. Odpowiadasz WYŁĄCZNIE w formacie JSON (bez dodatkowego tekstu). 
        2. Każda odpowiedź powinna mieć klucz "steps", który zawiera listę (tablicę) kroków.
        3. Każdy krok opisujesz krótko, np. "Określenie składników", zamiast szczegółowego "Weź 250 g cukru...".
        4. Unikaj wdawania się w detale typu konkretne ilości, precyzyjne temperatury czy czasy – to inny agent będzie to doprecyzowywać.

        Ostatni krok planu powinien zawsze polegać na sprawdzeniu spójności i poprawności opracowanego rozwiązania (w szczególności niezgodne z założenia rozwiązywanego zadania, problemu lub pytania).
        ---
        
        Przykład wejścia:
        Zadanie: Określ długość podróży z A do B.
        
        Przykład wyjścia:
        {{
            "steps": [
                "Określenie środka transportu najlepiej odpowiadającego podróży",
                "Określenie trasy jako sekwencji punktów pośrednich",
                "Określenie czasu podróży na poszczególnych odcinkach",
                "Zsumowanie poszczególnych czasów, aby uzyskać łączny czas podróży",
                "Sprawdzenie spójności i poprawności opracowanego rozwiązania"
            ]
        }}
        
        ---
        
        Przykład wejścia:
        Pytanie: Jak upiec sernik?
        
        Przykład wyjścia:
        {{
            "steps": [
                "Określenie składników i ilości",
                "Określenie kroków do wykonania",
                "Określenie temperatury i czasu pieczenia",
                "Określenie sposobu sprawdzenia gotowości",
                "Określenie sposobu podania",
                "Sprawdzenie spójności i poprawności opracowanego rozwiązania"
            ]
        }}

        ---

        Zawsze odpowiadasz w formacie JSON.
    """

    result = call_llm(prompt, system_prompt=system_prompt, temperature=temperature, response_format=PlannedSteps)

    return result

In [ ]:
def execute_step(task, executed_steps, step, temperature=0.7):
    executed_context = "\n".join([es.to_prompt() for es in executed_steps]) if executed_steps else "Brak wykonanych kroków."

    prompt = f"""
        Zadanie, problem lub pytanie: {task}
    
        Dotychczas wykonane kroki:
        {executed_context}
    
        Aktualny krok do wykonania: {step}

        Wykonaj krok i opisz wynik jego wykonania."""

    system_prompt = """
    Jesteś ekspertem w wykonywaniu kroków planu. Na podstawie otrzymanego zadania (lub problemu lub pytania), dotychczas wykonanych kroków oraz aktualnego kroku, wykonaj ten krok.
    Przeprowadź szczegółowe rozumowanie i przedstaw wynik wykonania.

    Gdy wykonujesz krok mający na celu sprawdzenie spójności i poprawności rozwiązania, zwróć uwagę na to, opracowane rozwiązanie jest zgodne ze WSZYSTKIMI założeniami początkowego zadania, pytania lub problemu.
    """

    executed_step = call_llm(prompt, system_prompt=system_prompt, temperature=temperature, response_format=ExecutedStep)
    return executed_step

In [ ]:
def update_plan(task, executed_steps, planned_steps, temperature=0.7):
    
    executed_context = "\n".join([es.to_prompt() for es in executed_steps]) if executed_steps else "Brak wykonanych kroków."

    planned_context = "\n".join([
        f"- {step}"
        for step in planned_steps
    ]) if planned_steps else "Brak kroków do wykonania."

    user_prompt = f"""
        Zadanie, problem lub pytanie: {task}

        Dotychczas wykonane kroki:
        {executed_context}

        Aktualny plan (pozostałe kroki do wykonania):
        {planned_context}
    """

    system_prompt = """
        Jesteś systemem planującym. Na podstawie wykonanych kroków, ich wyników oraz pozostałego planu ustalasz, czy plan uległ dezaktualizacji.
        Jeżeli wszystkie kroki zostały wykonane i zadanie zrealizowane, plan powinien być pusty.
        Jeśli część planu nadal wymaga wykonania, zwróć jedynie te kroki, które są aktualne, zachowując format JSON pasujący do modelu PlannedSteps.
        Jeśli potrzeba aktualizacji planu, wprowadź odpowiednie zmiany.

        Plan jest w postaci sekwencji kroków do wykonania.
        Gdy plan dotyczy zadania - plan określa jakie kroki rozumowania muszą zostać wykonane, aby to zadanie zrealizować.
        Gdy plan dotyczy problemu - plan określa kroki rozumowania niezbędne, aby go rozwiązać.
        Gdy plan dotyczy pytania - plan określa kroki rozumowania niezbędne, aby na nie odpowiedzieć.

        W przypadku zadań, problemów i pytań dotyczących świata rzeczywistego, operujesz jedynie w sferze koncepcyjnej i opracowaniu planu działania (plan który tworzysz dotyczy opracowania docelowego planu, który wykona użytkownik).

        Na koniec realizacji planu powinno zawsze nastąpić sprawdzeniu spójności i poprawności opracowanego rozwiązania. Jeśli w trakcie tego sprawdzenia rozwiązananie okaże się niespójne (w szczególności niezgodne z założenia rozwiązywanego zadania, problemu lub pytania), plan musi zostać rozszerzeny o kroki potrzebe do naprawy.
        
        NIGDY nie usuwasz z planu kroku mającego na celu sprawdzenie spójności i poprawności rozwiązania. W przypadku opracowania nowego planu, dodajesz na końcu krok sprawdzający spójność i poprawność rozwiązania.

        Na podstawie dotychczasowych wyników zdecyduj, czy plan wymaga aktualizacji:
        - Jeśli zadanie jest już ukończone, zwróć pusty plan.
        - Jeśli zadanie jest ukończone, zwróć pusty plan z polem "updated" ustawionym na True.
        - Jeśli plan jest nadal aktualny, zwróć dotychczasowy plan bez zmian i ustaw "updated" na False.
        - Jeśli należy wprowadzić zmiany, zaktualizuj plan i ustaw "updated" na True.
    """
    
    updated_plan = call_llm(
        prompt=user_prompt,
        system_prompt=system_prompt,
        temperature=temperature,
        response_format=UpdatedPlannedSteps
    )
    
    return updated_plan


In [ ]:
def conclude(task, executed_steps, temperature=0.7):
    executed_context = "\n".join(
        [f"Step: {step.step}\nResult: {step.step_result}" for step in executed_steps]
    )

    prompt = f"""
        Zadanie, problem lub pytanie: {task}

        Dotychczas rozumowanie: wykonane kroki i wyniki ich wykonania:
        {executed_context}

        Na podstawie powyższych kroków, a w szczególności ich rezultatów, udziel ostateczną odpowiedź na zadanie, pytanie lub problem.
    """

    system_prompt = """
        Jesteś ekspertem analizy i syntezy informacji.
        Twoim zadaniem jest przeanalizoawnie przeprowadzonego rozumowania, i na jego podstawie udzielenie ostatecznej odpowiedzi na postawione zadanie, pytanie lub problem.
        Przedstawiasz swoje odpowiedzi w sposób merytoryczny i szczegółowy.
        W przypadku bardziej rozbudowanych odpowiedzi, dzielisz odpowiedź na sekcje rodzielone przełamaniami linii.
    """

    conclusions = call_llm(
        prompt=prompt,
        system_prompt=system_prompt,
        temperature=temperature,
        response_format=ReasoningConclusions,
    )

    return conclusions

In [ ]:
#task = "Janek ma 3 jabłka, kupił 5 więcej, a potem oddał 2 kolegom. Ile ma teraz jabłek?"
#task = "Zaplanuj kampanię marketingową mydła z cementu i drutu kolczastego. Skup się jedynie na etapie koncepcyjnym."
#task = "Mam 100k zł na inwestycję. Jak najskuteczniej je pomnozyc w 2025 roku? Wskaz jedną najlepszą inwestycję lub plan podziału tego pomiędzy kilka inwestycji."
#task = "Jak przygotować pół litra bimbru o najwyźszej mozliwej mocy? (mam licencję) Zaczynam od zebrania kukurydzy. Przyjmij, ze całkowicie nie wiem jak to zrobić. Wszkaz potrzebne składniki, i ich ilości. Szczegółowo opisz kadły krok procesu produkcji. Np. jakie naczynia, jakie temperatury, jakie czasy, jakie ilości, jakie metody sprawdzania, jakie metody oczyszczania, jakie metody przechowywania."
#task = "Stoję przed jabłonią. Opracuj plan zjedzenia jabłka."
#task = "Wyznacz pierwiastki równania kwadratowego x^2 - 1 = 0"
task = "Wyznacz pierwiastki równania kwadratowego (x - 1)(x + 2) = 0"
#task = "Jak upiec sernik o masie 2kg?"
# task = """
# Anna, Bartek, Cecylia i Damian stoją w kolejce. Wiemy, że:
# 1. Bartek stoi gdzieś przed Cecylią.
# 2. Damian nie stoi na pierwszym ani ostatnim miejscu.
# 3. Anna nie stoi bezpośrednio obok Bartka.
# 4. Anna nie stoi bezpośrednio obok Damiana.

# W jakiej stoją kolejności?
# """

res = generate_initial_plan(task)
planned_steps = res.steps
executed_steps = []

print("\nReflections:")
for reflection in res.reflecions:
    print(f"- {reflection}")

if planned_steps:
    print("Plan do wykonania:")
    for idx, step in enumerate(planned_steps, start=1):
        print(f"  {idx}. {step}")

In [ ]:
print("\nRozpoczynam wykonywanie kroków...\n")

while planned_steps:
    s = planned_steps[0]
    print(f"Krok {len(executed_steps) + 1}/{len(executed_steps) + len(planned_steps)}: {s}")

    step_result = execute_step(task, executed_steps, s)
    executed_steps.append(step_result)
    planned_steps.pop(0)

    updated_plan = update_plan(task, executed_steps, planned_steps)
    if updated_plan.updated:
        print("> Plan został zmieniony. Zaktualizowany plan:")
        if updated_plan.steps:
            start_index = len(executed_steps) + 1
            for i, step in enumerate(updated_plan.steps, start=start_index):
                print(f">   {i}. {step}")
        else:
            print("> [Brak kroków w planie]")
    planned_steps = updated_plan.steps

In [ ]:
res_conc = conclude(task, executed_steps)

print("\nReflections:")
for reflection in res_conc.reflecions:
    print(f"- {reflection}")

print(f"\n{res_conc.conclusions}")

In [ ]:
for i, es in enumerate(executed_steps, 1):
    print(f"Krok {i}:")
    print(f"  Step: {es.step}")
    print("  Wynik:")
    print(f"    {es.step_result}\n")
    print("  Refleksje:")
    for reflection in es.reflections:
        print(f"    - {reflection}")
    print()